# Download Datasets

In [0]:
%sh 
curl -O 'https://raw.githubusercontent.com/masfworld/datahack_docker/master/zeppelin/data/Coral_cover_data.csv'
curl -O 'https://raw.githubusercontent.com/masfworld/datahack_docker/master/zeppelin/data/departuredelays.csv'
curl -O 'https://raw.githubusercontent.com/masfworld/datahack_docker/master/zeppelin/data/loan.csv'
curl -O 'https://raw.githubusercontent.com/masfworld/datahack_docker/master/zeppelin/data/OnlineRetail.csv'

In [0]:
dbutils.fs.mkdirs("/dataset")
dbutils.fs.cp('file:/databricks/driver/Coral_cover_data.csv','dbfs:/dataset/Coral_cover_data.csv')
dbutils.fs.cp('file:/databricks/driver/departuredelays.csv','dbfs:/dataset/departuredelays.csv')
dbutils.fs.cp('file:/databricks/driver/loan.csv','dbfs:/dataset/loan.csv')
dbutils.fs.cp('file:/databricks/driver/OnlineRetail.csv','dbfs:/dataset/OnlineRetail.csv')

# ACID Transactions

---



## Example 1

Atomicity

In [0]:
#Job-1
FILE_PATH = "/mnt/delta/atomicity"
dbutils.fs.rm(FILE_PATH,recurse=True)

data = spark.range(100, 200)
data.write.format("delta").mode("overwrite").save(FILE_PATH)

In [0]:
display(data)

In [0]:
dbutils.fs.ls("/mnt/delta/atomicity/")

In [0]:
dbutils.fs.head("/mnt/delta/atomicity/_delta_log/00000000000000000000.json")

In [0]:
data.display()

In [0]:
dbutils.fs.ls("/mnt/delta/atomicity/")

In [0]:
data.display()

In [0]:
spark.read.format("delta").load(FILE_PATH).count()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def atomicity(x):
  if (x == 60):    
    raise Exception("Error in: " + x)
  else: 
    return x

atomicity_udf = udf(atomicity, IntegerType())
  
try:
  spark.range(50,200) \
    .withColumn("id", atomicity_udf(col("id"))) \
    .write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(FILE_PATH)
except:
  print('Failed')  

In [0]:
spark.read.format("delta").load(FILE_PATH).count()

## Example 2

* Converting data to `delta` format
* Updates

In [0]:
%sql
DROP TABLE IF EXISTS coral

In [0]:
CORAL_DELTA_PATH = "/mnt/delta/coral"
dbutils.fs.rm(CORAL_DELTA_PATH, recurse=True)

In [0]:
dbutils.fs.ls(CORAL_DELTA_PATH)

In [0]:
from delta.tables import *
from pyspark.sql.functions import *

In [0]:
coral_df = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .csv("/dataset/Coral_cover_data.csv")

coral_df.write.format("delta").save(CORAL_DELTA_PATH)
spark.sql("CREATE TABLE coral USING DELTA LOCATION \'" + CORAL_DELTA_PATH + "\'")

In [0]:
%sql
DROP TABLE IF EXISTS coral_2

In [0]:

dbutils.fs.rm("/user/hive/warehouse/coral_2", recurse=True)

In [0]:
coral_df.write.format("delta").saveAsTable("coral_2")

In [0]:
%sql
SELECT *
FROM
  coral
LIMIT 10

In [0]:
%sql
UPDATE coral SET Percent_cover = 0.0

In [0]:
dbutils.fs.ls("/mnt/delta/coral/")

In [0]:
dbutils.fs.head("/mnt/delta/coral/_delta_log/00000000000000000001.json")

In [0]:
%sql
SELECT *
FROM
  coral
LIMIT 10

## Exercise 1
---

Using Delta Table API (No SQL):
1. Creating a *DeltaTable* object, loading data from example 2 (Coral Dataset).
2. Update table with `Percent_cover = 1` where `Taxon` is equal to `Acropora_cervicornis` (Using Delta Table API)

*Hint: https://docs.delta.io/latest/api/python/spark/index.html*

In [0]:
from delta.tables import *
from pyspark.sql.functions import *

# Time Travel

## Example 3

In [0]:
%sql
DESCRIBE HISTORY coral

In [0]:
spark.read \
  .format("delta") \
  .option("timestampAsOf", "2024-03-20T19:03:20") \
  .load(CORAL_DELTA_PATH) \
  .display()

## Exercise 2
---

Do the following tasks:
1. Read `departuredelays.csv` file into a Dataframe
2. Convert the previous DataFrame in a Delta table
3. Build an example query over the table
4. Get the history of the delta table
5. Update the column `delay` to 0, and check changes
6. Restore the table to the initial state

In [0]:
tripdelaysFilePath = "/dataset/departuredelays.csv"

In [0]:
%sql
DROP TABLE IF EXISTS flights

In [0]:

dbutils.fs.rm("/user/hive/warehouse/flights", recurse=True)

## Exercise 3
---
Do the following tasks (using SQL and Delta Table API):
1. Delete rows where `origin = ORD` (Using Delta Table API)
2. Check the results
2. Restore table to the original state

# Schema Evolution

## Example 4

In [0]:
loans_df = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .csv('/dataset/loan.csv')

In [0]:
loans_df.display()

In [0]:
loans_ver = loans_df.select("loan_amnt", "verification_status")
loan_by_ver = loans_ver.groupBy("verification_status").count()

loan_by_ver.createOrReplaceTempView("loan_by_ver")

In [0]:
%sql
SELECT * FROM loan_by_ver LIMIT 10

In [0]:
LOANS_DELTA_PATH = "/mnt/delta/loans"
dbutils.fs.rm(LOANS_DELTA_PATH,recurse=True)

In [0]:
%sql
DROP TABLE IF EXISTS loan_by_ver_delta

In [0]:
%sql 
CREATE TABLE loan_by_ver_delta USING delta LOCATION '/mnt/delta/loans' AS
SELECT
  *
FROM
  loan_by_ver

In [0]:
%sql
SELECT * FROM loan_by_ver_delta

In [0]:
loans = spark.sql("""
            SELECT verification_status, CAST(rand(10)*count as bigint) AS count,
            CAST(rand(10) * 10000 * count AS double) AS amount
            FROM loan_by_ver_delta
            """)


In [0]:
loans.display()

In [0]:
loans.printSchema()

In [0]:
loan_by_ver.printSchema()

In [0]:
loans.write.format("delta") \
           .mode("append") \
           .save('/mnt/delta/loans')

In [0]:
# Add the mergeSchema option
loans.write.format("delta") \
           .option("mergeSchema", "true") \
           .mode("append") \
           .save('/mnt/delta/loans')

In [0]:
%sql
SELECT * FROM loan_by_ver_delta

In [0]:
%sql
SELECT
  verification_status,
  sum(amount) AS amount_sum
FROM
  loan_by_ver_delta
GROUP BY
  verification_status
ORDER BY
  sum(amount)

In [0]:
from pyspark.sql.functions import *

# Create loan by state
loan_by_ver_amnt = loans_df.groupBy("verification_status").agg(avg('loan_amnt').alias('avg_amnt'))
loan_by_ver_amnt.display()

In [0]:
loan_by_ver_amnt.write.format("delta") \
           .mode("append") \
           .save('/mnt/delta/loans')

In [0]:
loan_by_ver_amnt.write.format("delta") \
           .option("overwriteSchema", "true") \
           .mode("overwrite") \
           .save('/mnt/delta/loans')

In [0]:
%sql
SELECT * FROM loan_by_ver_delta

In [0]:
%sql
DESCRIBE HISTORY loan_by_ver_delta

In [0]:
spark.read \
  .format("delta") \
  .option("versionAsOf", 1) \
  .load('/mnt/delta/loans') \
  .display()

## Exercise 4
---
Do the following tasks over flight table built in Time Travel section:
1. Get the average of distances for each origin airport. Store the result in a new delta table called `flight_grouped`
2. Create new Delta Table/ Dataframe grouping flights data by origin with only the average of the delay
3. Merge the information generated in point 1 and 2 in `flight_grouped` table. We want to get a table with the following information:
  *   Origin Airport
  *   Distance Average
  *   Delay Average
4. Restore `flight_grouped` table to the previous version

In [0]:
dbutils.fs.rm("/user/hive/warehouse/flight_grouped",recurse=True)

In [0]:
%sql
drop table if exists flight_grouped

# Constraints

## Exercise 5
---
Using SQL:
1. Create a delta table called `flights_constraint` with the same schema than `flights`(Used in exercise 2 and 3). `flights_constraint` table must have `destination`column as `NOT NULL`. 
2. Insert a test row with `destination` column with `NULL`.
3. Remove "NOT NULL" constraint in "destination" column. Check again point 2
4. Add constraint `(delay > -10000)`. Check it with an insert statement.

# Upsert with Merge

## Example 5

In [0]:
miniDataInputPath = "/dataset/OnlineRetail.csv"
basePath            = "/mnt/delta/retail"
deltaMiniDataPath   = basePath + "/customer-data-mini/"

inputSchema = "InvoiceNo INT, StockCode STRING, Description STRING, Quantity INT, InvoiceDate STRING, UnitPrice DOUBLE, CustomerID INT, Country STRING"

In [0]:
miniDataDF = spark.read \
  .option("header", "true") \
  .schema(inputSchema) \
  .csv(miniDataInputPath)

miniDataDF \
  .write \
  .mode("overwrite") \
  .format("delta") \
  .save(deltaMiniDataPath)

In [0]:
miniDataDF.display()

In [0]:
%sql
DROP TABLE IF EXISTS customer_data_delta_mini

In [0]:
spark.sql("""
    CREATE TABLE customer_data_delta_mini
    USING DELTA 
    LOCATION '{}' 
  """.format(deltaMiniDataPath))

In [0]:
%sql
SELECT * FROM customer_data_delta_mini  LIMIT 10

In [0]:
%sql
SELECT * FROM customer_data_delta_mini WHERE CustomerID=17925

In [0]:
dbutils.fs.rm("/user/hive/warehouse/customer_data_delta_to_upsert",recurse=True)

In [0]:
from pyspark.sql.functions import lit, col

customerSpecificDF = (miniDataDF
  .filter("CustomerID=17925")
  .withColumn("StockCode", lit(99999))
  .withColumn("InvoiceNo", col("InvoiceNo").cast("String")) 
 )

spark.sql("DROP TABLE IF EXISTS customer_data_delta_to_upsert")
customerSpecificDF.write.saveAsTable("customer_data_delta_to_upsert")

In [0]:
%sql
DESCRIBE HISTORY customer_data_delta_to_upsert

In [0]:
%sql
SELECT * FROM customer_data_delta_to_upsert

In [0]:
%sql
SELECT * FROM customer_data_delta_mini WHERE CustomerID=17925

In [0]:
%sql
MERGE INTO customer_data_delta_mini
USING customer_data_delta_to_upsert
ON customer_data_delta_mini.CustomerID = customer_data_delta_to_upsert.CustomerID
WHEN MATCHED THEN
  UPDATE SET
    customer_data_delta_mini.StockCode = customer_data_delta_to_upsert.StockCode
WHEN NOT MATCHED
  THEN INSERT (InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country)
  VALUES (
    customer_data_delta_to_upsert.InvoiceNo,
    customer_data_delta_to_upsert.StockCode, 
    customer_data_delta_to_upsert.Description, 
    customer_data_delta_to_upsert.Quantity, 
    customer_data_delta_to_upsert.InvoiceDate, 
    customer_data_delta_to_upsert.UnitPrice, 
    customer_data_delta_to_upsert.CustomerID, 
    customer_data_delta_to_upsert.Country)

In [0]:
%sql
SELECT * FROM customer_data_delta_mini WHERE CustomerID=17925

In [0]:
%sql
DESCRIBE HISTORY customer_data_delta_mini

## Exercise 6
---

Do the following tasks:
1. Generate a delta table called `flights_grouped_by_origin` from DataFrame with the average of delay grouped by `origin`, built in exercise 2
2. Generate a delta table called `flights_grouped` calculating the maximum delay for each origin airport
3. Merge (using SQL) the table built in the point 1 and table restored in exercise 2

*Hints: https://docs.delta.io/latest/delta-update.html#language-python </br>
https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/delta-merge-into * 


In [0]:
%sql
SET spark.databricks.delta.properties.defaults.enableChangeDataFeed = true

In [0]:
%sql
SET spark.databricks.delta.schema.autoMerge.enabled = true

# Change Data Feed

## Exercise 7
---

Do the following tasks:
1. Capture all changes coming from `flights_grouped_by_origin` delta table
2. Write these rows to `flights_delay` topic in Kafka
3. Read data from previous topic, filter out by a random origin airport, and saving the result in another delta table.

In [0]:
%sql
SET spark.databricks.delta.properties.defaults.enableChangeDataFeed = true

In [0]:
dbutils.fs.rm("/user/hive/warehouse/flights_grouped_by_origin",recurse=True)

In [0]:
%sql
DROP TABLE IF EXISTS flights_grouped_by_origin